<a href="https://colab.research.google.com/github/soumeyademil/Face-Mask-Detection/blob/main/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face-Mask-Detection
### Fait par : DEMIL et GHAOUAT
### Groupe : 2SIQ 2

# Modèle : Preprocessing + Training + Tests

## Préparation des données

*   Le dataset est téléchargé à partir de Kaggle https://www.kaggle.com/omkargurav/face-mask-dataset.
*   On utilise drive pour stocker les images.
*   On divisera ensuite les données sur 3 repertoires : train_dir (80%), validation_dir (10%), test_dir (10%).






In [1]:
#Configuration environment
'''
import os

os.environ['KAGGLE_USERNAME'] = "soumeyademil" # username from the json file
os.environ['KAGGLE_KEY'] = "61ee9cfa1c91fb28180125ffa84acea4" # key from the json file

!kaggle datasets download -d omkargurav/face-mask-dataset

import zipfile
with zipfile.ZipFile("/content/face-mask-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/Kaggle/face_mask_data")

'''    

'\nimport os\n\nos.environ[\'KAGGLE_USERNAME\'] = "soumeyademil" # username from the json file\nos.environ[\'KAGGLE_KEY\'] = "61ee9cfa1c91fb28180125ffa84acea4" # key from the json file\n\n!kaggle datasets download -d omkargurav/face-mask-dataset\n\nimport zipfile\nwith zipfile.ZipFile("/content/face-mask-dataset.zip", \'r\') as zip_ref:\n    zip_ref.extractall("/content/drive/MyDrive/Kaggle/face_mask_data")\n\n'

In [2]:
'''
import splitfolders
input_folder='/content/drive/MyDrive/Kaggle/face_mask_data'
output='/content/drive/MyDrive/Kaggle/face_mask_data_processed'
splitfolders.ratio(input_folder, output, seed=42, ratio=(.8, .1, .1))
'''

"\nimport splitfolders\ninput_folder='/content/drive/MyDrive/Kaggle/face_mask_data'\noutput='/content/drive/MyDrive/Kaggle/face_mask_data_processed'\nsplitfolders.ratio(input_folder, output, seed=42, ratio=(.8, .1, .1))\n"

In [3]:
base_path = '/content/drive/MyDrive/Kaggle/face_mask_data_processed/'

PATH_TRAIN_MASK = base_path + 'train/with_mask/'
PATH_VAL_MASK = base_path + 'val/with_mask/'
PATH_TEST_MASK = base_path + 'test/with_mask/'

PATH_TRAIN_NO_MASK = base_path + 'train/without_mask/'
PATH_VAL_NO_MASK = base_path + 'val/without_mask/'
PATH_TEST_NO_MASK = base_path + 'test/without_mask/'

PATH_TRAIN = base_path + 'train/'
PATH_VAL = base_path + 'val/'
PATH_TEST = base_path + 'test/'

Dans cette cellule on determine quelques paramètres et on normalise les données, pour ensuite créer des objets ImageDataGenerator qui vont permettre au modèle de récupérer les images à partir des repertoires directement.







In [4]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32
target_size = (200, 200)
num_classes = 2
num_epochs = 10


datagen_train = ImageDataGenerator(rescale=1/255)
datagen_val = ImageDataGenerator(rescale=1/255)
datagen_test = ImageDataGenerator(rescale=1/255)

train_generator = datagen_train.flow_from_directory(PATH_TRAIN,
                                                    target_size=target_size,
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = datagen_val.flow_from_directory(PATH_VAL,
                                                    target_size=target_size,
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='binary')

test_generator = datagen_test.flow_from_directory(PATH_TEST,
                                                    target_size=target_size,
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    shuffle=False)

Found 6042 images belonging to 2 classes.
Found 754 images belonging to 2 classes.
Found 757 images belonging to 2 classes.


In [5]:
train_generator.class_indices

{'with_mask': 0, 'without_mask': 1}

Pour le modèle, on utilisera le transfert learning, avec le modèle ResNet50 qui est un CNN qui a été entrainé sur un million d'images. On lui ajoutera une couche dense de 10 unités cachées, et une couche de sortie avec une unité utilisant la fonction d'activation sigmoid qui est la plus adapté pour la classification binaire.

In [6]:
import tensorflow as tf

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
md = ResNet50(include_top=False, input_shape=(200, 200, 3))
input_layer = Input(shape=(200, 200, 3), name = 'input')
layer1 = md(input_layer)
layer2 = Flatten()(layer1)
layer3 = Dense(10, activation='relu')(layer2)
output_layer = Dense(1, activation='sigmoid')(layer3)

model = Model(inputs = [input_layer], outputs = output_layer)


94781440/94765736 [==============================] - 0s 0us/step


In [8]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

In [ ]:
checkpoint = ModelCheckpoint("/content/drive/MyDrive/TP_IMN_Face_Mask/model_weights.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history = model.fit(train_generator, batch_size=batch_size, epochs=num_epochs, validation_data=val_generator, callbacks=callbacks_list)

Epoch 1/10
  2/189 [..............................] - ETA: 52:59 - loss: 2.4066 - acc: 0.4219  

On a obtenu une précision de 94% sur le validation set, les paramètres correspondant à cette valeur(max) sont enregistrés dans un fichier.h5.

In [ ]:
model_json = model.to_json()
with open("/content/drive/MyDrive/TP_IMN_Face_Mask/model.json", "w") as json_file:
    json_file.write(model_json)

La matrice de confusion permet de visualiser la qualité de classification du modèle.

In [ ]:

from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

model = load_model('/content/drive/MyDrive/TP_IMN_Face_Mask/model_weights.h5')
predictions = model.predict_generator(test_generator)
y_pred = (predictions > 0.5)
y_test = test_generator.classes
class_names = test_generator.class_indices.keys()

from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()

In [ ]:
loss, acc = model.evaluate(test_generator) 
print("Test accuracy = " , acc)

In [ ]:
from keras.models import model_from_json
import numpy as np

class FaceMaskModel(object):

    CLASSES_LIST = ["WITH MASK",
                    "WITHOUT MASK"]

    def __init__(self, model_json_file, model_weights_file):
        # load model from JSON file
        with open(model_json_file, "r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)

        # load weights into the new model
        self.loaded_model.load_weights(model_weights_file)
        #self.loaded_model._make_predict_function()

    def predict_class(self, img):
        self.preds = self.loaded_model.predict(img)
        return FaceMaskModel.CLASSES_LIST[self.preds > 0.5]

In [ ]:
import tensorflow
tensorflow.__version__

# Reconnaissance en temps reel

Dans cette partie, on utilise le framework Flask qui va permettre de créer une interface graphique web pour notre application.

In [ ]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import numpy as np
import cv2

In [ ]:
CLASSES_LIST = ["Avec masque",
                "Sans masque!"]

beep_duration = 1000  
beep_freq = 440  

facec = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
model = load_model("Face_Mask_Model//model_weights.h5")
font = cv2.FONT_HERSHEY_SIMPLEX

class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(0)

    def __del__(self):
        self.video.release()

    # returns camera frames along with bounding boxes and predictions

    def get_frame(self):
        _, fr = self.video.read()
        faces = facec.detectMultiScale(fr)

        for (x, y, w, h) in faces:
            fc = fr[y:y+h, x:x+w]

            roi = cv2.resize(fc, (200, 200))
            roi = roi / 255.
            pred = model.predict(roi[np.newaxis, :, :, :])
            cond = (pred < 0.5)
            text = CLASSES_LIST[int(cond)]
            
            if (cond) :
                cv2.putText(fr, text, (x, y), font, 1, (0, 255, 255), 2)
                cv2.rectangle(fr,(x,y),(x+w,y+h),(0,0,255),2)
                winsound.Beep(beep_freq, beep_duration)
            else:
                cv2.putText(fr, text, (x, y), font, 1, (255, 255, 0), 2)
                cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        _, jpeg = cv2.imencode('.jpg', fr)
        return jpeg.tobytes()

In [ ]:
import camera
from flask import Flask, render_template, Response
import os
#from camera import VideoCamera

app = Flask(__name__, template_folder='template')

#os.environ['FLASK_APP'] = app
os.environ['FLASK_ENV'] = 'development'



@app.route('/')
def index():
    return render_template('index.html')

def gen(camera):
    while True:
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(gen(VideoCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)